# Packages

In [1]:
import pandas as pd
import numpy as np
import os, re, csv, codecs, operator, sys, gc
from collections import defaultdict, OrderedDict
from tqdm import tqdm
from os import listdir
from os.path import isfile, join
import lightgbm as lgb
from matplotlib import pyplot as plt
%matplotlib inline

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score
from sklearn.model_selection import cross_val_score, cross_val_predict, StratifiedKFold, KFold, train_test_split
from sklearn.ensemble import ExtraTreesClassifier

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from string import punctuation
from textblob import TextBlob    # For pos-tagging
from scipy.sparse import csr_matrix, hstack
from scipy.special import logit, expit

from keras import optimizers, initializers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Lambda, Embedding, Dropout, Activation, SpatialDropout1D, Reshape, \
GlobalAveragePooling1D, merge, Flatten, Bidirectional, CuDNNGRU, add, Conv1D, GlobalMaxPooling1D
from keras.layers.merge import concatenate
from keras.models import Model, Sequential
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.engine import InputSpec, Layer
from keras import backend as K

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Parameters

In [2]:
PATH = 'datasets/'
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
FAST_TEXT_EMBEDDING = 'pretrain_embedding/crawl-300d-2M.vec'
GLOVE_EMBEDDING = 'pretrain_embedding/glove.840B.300d.txt'
CLEAN_WORD_PATH = None
TRAIN_DATA_FILE = 'train.csv'
TEST_DATA_FILE = 'test.csv'
MAX_SEQUENCE_LENGTH = 350
MAX_NB_WORDS = 100000
EMBEDDING_DIM = 300
FOLD_COUNT = 10
BATCH_SIZE = 200

# Load Pretrain Models

In [3]:
def load_pretrain_embedding(file):
    print('Indexing word vectors')
    embeddings_index = {}
    f = open(file, 'r', encoding='utf-8')
    for line in f:
        values = line.split()
        try:
            word = values[0]
            coefs = np.array(values[1:], dtype='float32')
            embeddings_index[word] = coefs
        except:
            print("Error on: ", values[:3])
    f.close()
    print("Total %s word vectors" % len(embeddings_index))
    return embeddings_index

In [4]:
embeddings_index = load_pretrain_embedding(FAST_TEXT_EMBEDDING)

Indexing word vectors
Total 2000000 word vectors


In [5]:
glove_embeddings_index = load_pretrain_embedding(GLOVE_EMBEDDING)

Indexing word vectors
Error on:  ['.', '.', '.']
Error on:  ['at', 'name@domain.com', '0.0061218']
Error on:  ['.', '.', '.']
Error on:  ['to', 'name@domain.com', '0.33865']
Error on:  ['.', '.', '0.035974']
Error on:  ['.', '.', '.']
Error on:  ['email', 'name@domain.com', '0.33529']
Error on:  ['or', 'name@domain.com', '0.48374']
Error on:  ['contact', 'name@domain.com', '0.016426']
Error on:  ['Email', 'name@domain.com', '0.37344']
Error on:  ['on', 'name@domain.com', '0.037295']
Error on:  ['At', 'Killerseats.com', '-0.13854']
Error on:  ['by', 'name@domain.com', '0.6882']
Error on:  ['in', 'mylot.com', '-0.18148']
Error on:  ['emailing', 'name@domain.com', '0.39173']
Error on:  ['Contact', 'name@domain.com', '0.14933']
Error on:  ['at', 'name@domain.com', '0.44321']
Error on:  ['•', 'name@domain.com', '-0.13288']
Error on:  ['at', 'Amazon.com', '-0.5275']
Error on:  ['is', 'name@domain.com', '-0.1197']
Total 2195885 word vectors


# Data Overview

In [6]:
train_df = pd.read_csv('datasets/train.csv')
test_df = pd.read_csv('datasets/test.csv')

In [7]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 8 columns):
id               159571 non-null object
comment_text     159571 non-null object
toxic            159571 non-null int64
severe_toxic     159571 non-null int64
obscene          159571 non-null int64
threat           159571 non-null int64
insult           159571 non-null int64
identity_hate    159571 non-null int64
dtypes: int64(6), object(2)
memory usage: 9.7+ MB


In [8]:
train_df.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000
mean,0.095844,0.009996,0.052948,0.002996,0.049364,0.008805
std,0.294379,0.099477,0.223931,0.054650,0.216627,0.093420
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [9]:
train_df.corr()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
toxic,1.000000,0.308619,0.676515,0.157058,0.647518,0.266009
severe_toxic,0.308619,1.000000,0.403014,0.123601,0.375807,0.201600
obscene,0.676515,0.403014,1.000000,0.141179,0.741272,0.286867
threat,0.157058,0.123601,0.141179,1.000000,0.150022,0.115128
insult,0.647518,0.375807,0.741272,0.150022,1.000000,0.337736
identity_hate,0.266009,0.201600,0.286867,0.115128,0.337736,1.000000


In [10]:
train_df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [11]:
train_comments = train_df['comment_text'].values
test_comments = test_df['comment_text'].values
train_comments_lengths = [len(s) for s in train_comments]
test_comments_lengths = [len(s) for s in test_comments]

In [12]:
def explore_comments(arr):
    print("MAX LENGTH:\t\t", np.max(arr))
    print("AVG LENGTH:\t\t", np.average(arr))
    print("MIN LENGTH:\t\t", np.min(arr))
    print("STANDARD DIVISION:\t", np.std(arr))
    print("RANGE:\t\t\t", np.min(arr), " to ", np.average(arr) + 2 * np.std(arr))
    
print("------Train------")
explore_comments(train_comments_lengths)

print("------Test------")
explore_comments(test_comments_lengths)

------Train------
MAX LENGTH:		 5000
AVG LENGTH:		 394.0732213246768
MIN LENGTH:		 6
STANDARD DIVISION:	 590.7184309382144
RANGE:			 6  to  1575.5100832011055
------Test------
MAX LENGTH:		 5000
AVG LENGTH:		 364.8751207855632
MIN LENGTH:		 1
STANDARD DIVISION:	 592.4901645516661
RANGE:			 1  to  1549.8554498888955


In [13]:
for class_name in list_classes:
    print('{}\n{}\n'.format(class_name, train_df[class_name].value_counts()))

toxic
0    144277
1     15294
Name: toxic, dtype: int64

severe_toxic
0    157976
1      1595
Name: severe_toxic, dtype: int64

obscene
0    151122
1      8449
Name: obscene, dtype: int64

threat
0    159093
1       478
Name: threat, dtype: int64

insult
0    151694
1      7877
Name: insult, dtype: int64

identity_hate
0    158166
1      1405
Name: identity_hate, dtype: int64



In [14]:
def count_regexp_occ(regexp='', text=None):
    """
    Simple way to calculate the number of occurence of a regex
    """
    if len(text) == 0:
        return 0
    else:
        return len(re.findall(regexp, text)) / len(text)

# Data Cleaning

## Load Cleaned Words

In [15]:
if CLEAN_WORD_PATH == None:
    ignored_words = set(stopwords.words('english'))
else:
    ignored_words = {}
    with open(CLEAN_WORD_PATH, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip('\n')
            typo, correct = line.split(',')
            ignored_words[typo] = correct

## Ensemble Features

In [16]:
feature_files = sorted([f for f in listdir('datasets/feature_files/') if isfile(join('datasets/feature_files/', f))])
for file in feature_files:
    fixed_df = pd.read_csv('datasets/feature_files/' + file)
    train_df = train_df.merge(fixed_df, on='id', how='left')
    test_df = test_df.merge(fixed_df, on='id',how='left')

In [17]:
def ensemble_data(path):
    files = sorted([f for f in listdir(path) if isfile(join(path, f))])
    for i, file in enumerate(files):
        temp_df = pd.read_csv(path + file)
        print('Datasets before ensemble:', temp_df.isnull().sum().sum())
        if i == 0:
            ensembled = temp_df
        else:
            ensembled = ensembled.merge(temp_df, on='id', how='left')
            print('Datasets after ensemble:', ensembled.isnull().sum().sum())
    
    column_numbers = ensembled.shape[1]
    toxic = ensembled.iloc[:, [i for i in range(2, column_numbers, len(list_classes))]]
    severe_toxic = ensembled.iloc[:, [i for i in range(3, column_numbers, len(list_classes))]]
    obscene = ensembled.iloc[:, [i for i in range(4, column_numbers, len(list_classes))]]
    threat = ensembled.iloc[:, [i for i in range(5, column_numbers, len(list_classes))]]
    insult = ensembled.iloc[:, [i for i in range(6, column_numbers, len(list_classes))]]
    identity_hate = ensembled.iloc[:, [i for i in range(7, column_numbers, len(list_classes))]]
    
    return toxic, severe_toxic, obscene, threat, insult, identity_hate

In [18]:
train_toxic, train_severe_toxic, train_obscene, train_threat, train_insult, train_identity_hate = ensemble_data('datasets/ensemble_files/train/')
test_toxic, test_severe_toxic, test_obscene, test_threat, test_insult, test_identity_hate = ensemble_data('datasets/ensemble_files/test/')

Datasets before ensemble: 0
Datasets before ensemble: 0


## Regex Regularization

In [19]:
# Regex to remove all Non-Alpha Numeric and space
special_character_removal = re.compile(r'[^?!.,:a-z\d ]', re.IGNORECASE)
# Regex to remove all numerics
replace_numbers = re.compile(r'\d+', re.IGNORECASE)

word_count_dict = defaultdict(int)

def clean_datasets(text, remove_stopwords=False, stem_words=False, count_null_words=True, clean_wiki_tokens=True):
    
    text = text.lower()
    text = re.sub(r"https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)", "", text)
    text = re.sub(r"”", "\"", text)
    text = re.sub(r"“", "\"", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"i’m", "i am", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = replace_numbers.sub('', text)
    
    if count_null_words:
        text = text.split()
        for t in text:
            word_count_dict[t] += 1
        text = " ".join(text)
        
    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)
        
    return text

In [20]:
list_sentences_train = train_df['comment_text'].fillna('no comment').values
list_sentences_test = test_df['comment_text'].fillna('no comment').values
train_labels = train_df[list_classes].values

In [21]:
cleaned_train_comments, cleaned_test_comments = [], []
print('Processing data cleaning...')

for text in list_sentences_train:
    cleaned_train_comments.append(clean_datasets(text))
for text in list_sentences_test:
    cleaned_test_comments.append(clean_datasets(text))
print('Done!')

Processing data cleaning...
Done!


In [22]:
train_df['comment_text_cleaned'] = cleaned_train_comments
test_df['comment_text_cleaned'] = cleaned_test_comments

## Create Meta (Extend) Features

In [23]:
def count_unknown_embedding(t, idx):
    t = t.split()
    res = 0
    for word in t:
        if word not in idx.keys():
            res += 1
    return res

In [24]:
def create_meta_feature(df):
    df['total_length'] = df['comment_text'].apply(len)
    df['capital'] = df['comment_text'].apply(lambda comment: sum(1 for c in comment if c.isupper()))
    df['caps_vs_length'] = df.apply(lambda row: float(row['capital']) / float(row['total_length']), axis=1)
    df["unknown_glove"] = df['comment_text_cleaned'].apply(lambda x: count_unknown_embedding(x, glove_embeddings_index))
    df["unknown_fasttext"] = df['comment_text_cleaned'].apply(lambda x: count_unknown_embedding(x, embeddings_index))
    df["unknown_glove_fasttext"] = df["unknown_glove"] + df["unknown_fasttext"]
    # Special Expressions Collection
    df['num_exclamation_marks'] = df['comment_text'].apply(lambda comment: comment.count('!'))
    df['num_question_marks'] = df['comment_text'].apply(lambda comment: comment.count('?'))
    df['num_punctuation'] = df['comment_text'].apply(lambda comment: sum(comment.count(w) for w in '.,;:'))
    df['num_symbols'] = df['comment_text'].apply(lambda comment: sum(comment.count(w) for w in '*&$%'))
    df['num_words'] = df['comment_text'].apply(lambda comment: len(comment.split()))
    df['num_unique_words'] = df['comment_text'].apply(lambda comment: len(set(w for w in comment.split())))
    df['words_vs_unique'] = df['num_unique_words'] / df['num_words']
    df['num_smilies'] = df['comment_text'].apply(lambda comment: sum(comment.count(w) for w in (':-)', ':)', ';-)', ';)')))
    df["ant_slash_n"] = df["comment_text"].apply(lambda x: count_regexp_occ(r"\n", x))
    df["start_with_columns"] = df["comment_text"].apply(lambda x: count_regexp_occ(r"^\:+", x))
    df["has_emphasize_equal"] = df["comment_text"].apply(lambda x: count_regexp_occ(r"\={2}.+\={2}", x))
    df["has_emphasize_quotes"] = df["comment_text"].apply(lambda x: count_regexp_occ(r"\"{4}\S+\"{4}", x))
    df["has_star"] = df["comment_text"].apply(lambda x: count_regexp_occ(r"\*", x))
    ## Check for dates
    df["has_timestamp"] = df["comment_text"].apply(lambda x: count_regexp_occ(r"\d{2}|:\d{2}", x))
    df["has_date_long"] = df["comment_text"].apply(lambda x: count_regexp_occ(r"\D\d{2}:\d{2}, \d{1,2} \w+ \d{4}", x))    # example: 18:44, 8 December 2010
    df["has_date_short"] = df["comment_text"].apply(lambda x: count_regexp_occ(r"\D\d{1,2} \w+ \d{4}", x))
    ## Check for http links
    df["has_http"] = df["comment_text"].apply(lambda x: count_regexp_occ(r"http[s]{0,1}://\S+", x))
    df["has_ip"] = df["comment_text"].apply(lambda x: count_regexp_occ("(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)(\.(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)){3}", x))
    ## Check for mail
    df["has_mail"] = df["comment_text"].apply(lambda x: count_regexp_occ(r'[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+', x))
    ## Check for image
    df["has_image"] = df["comment_text"].apply(lambda x: count_regexp_occ(r'image\:', x))
    # Dirty Languages Collection
    df["nb_fk"] = df["comment_text"].apply(lambda x: count_regexp_occ(r"[Ff]\S{2}[Kk]", x))
    df["nb_sk"] = df["comment_text"].apply(lambda x: count_regexp_occ(r"[Ss]\S{2}[Kk]", x))
    df["nb_dk"] = df["comment_text"].apply(lambda x: count_regexp_occ(r"[dD]ick", x))
    df["nb_you"] = df["comment_text"].apply(lambda x: count_regexp_occ(r"\W[Yy]ou\W", x))
    df["nb_mother"] = df["comment_text"].apply(lambda x: count_regexp_occ(r"\Wmother\W", x))
    df["nb_ng"] = df["comment_text"].apply(lambda x: count_regexp_occ(r"\Wnigger\W", x))

In [25]:
print("Creating meta features...")
create_meta_feature(train_df)
create_meta_feature(test_df)
print('Done!')

Creating meta features...
Done!


In [26]:
train_df = train_df.drop(['comment_text'], axis=1)
test_df = test_df.drop(['comment_text'], axis=1)

In [27]:
train_cols = train_df.columns[8:]
print(train_cols)
column_numbers = len(train_df.columns)
print(column_numbers, len(train_cols))

Index(['total_length', 'capital', 'caps_vs_length', 'unknown_glove',
       'unknown_fasttext', 'unknown_glove_fasttext', 'num_exclamation_marks',
       'num_question_marks', 'num_punctuation', 'num_symbols', 'num_words',
       'num_unique_words', 'words_vs_unique', 'num_smilies', 'ant_slash_n',
       'start_with_columns', 'has_emphasize_equal', 'has_emphasize_quotes',
       'has_star', 'has_timestamp', 'has_date_long', 'has_date_short',
       'has_http', 'has_ip', 'has_mail', 'has_image', 'nb_fk', 'nb_sk',
       'nb_dk', 'nb_you', 'nb_mother', 'nb_ng'],
      dtype='object')
40 32


In [28]:
for col in train_cols:
    train_toxic[col] = train_df[col]
    train_severe_toxic[col] = train_df[col]
    train_obscene[col] = train_df[col]
    train_threat[col] = train_df[col]
    train_insult[col] = train_df[col]
    train_identity_hate[col] = train_df[col]
for col in train_cols:
    test_toxic[col] = test_df[col]
    test_severe_toxic[col] = test_df[col]
    test_obscene[col] = test_df[col]
    test_threat[col] = test_df[col]
    test_insult[col] = test_df[col]
    test_identity_hate[col] = test_df[col]
train_ensembled_data = [train_toxic, train_severe_toxic, train_obscene, train_threat, train_insult, train_identity_hate]
test_ensembled_data = [test_toxic, test_severe_toxic, test_obscene, test_threat, test_insult, test_identity_hate]

In [29]:
del list_sentences_train, list_sentences_test
gc.collect()

280

# Data Preprocessing

In [30]:
all_comment_text = pd.concat([train_df['comment_text_cleaned'], test_df['comment_text_cleaned']], axis=0).fillna("unknown")
nrow_train = train_df.shape[0]
all_comment_text.shape[0]

312735

In [31]:
vectorizer = TfidfVectorizer(stop_words='english', max_features=50000)
data = vectorizer.fit_transform(all_comment_text)
print(data.shape)

(312735, 50000)


/usr/local/lib/python3.5/dist-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [32]:
norm_data = MaxAbsScaler().fit_transform(data)
print(norm_data.shape)

(312735, 50000)


## Tf-idf Feature Extraction

## Word-Level

In [33]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    ngram_range=(1, 1),
    max_features=20000
)
word_vectorizer.fit(all_comment_text)
train_word_features = word_vectorizer.transform(cleaned_train_comments)
test_word_features = word_vectorizer.transform(cleaned_test_comments)
print('Word vectorization process Done!')

/usr/local/lib/python3.5/dist-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


Word vectorization process Done!


## Char-Level

In [34]:
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    ngram_range=(1, 6),
    max_features=30000
)
char_vectorizer.fit(all_comment_text)
train_char_features = char_vectorizer.transform(cleaned_train_comments)
test_char_features = char_vectorizer.transform(cleaned_test_comments)
print('Char vectorization process Done!')

/usr/local/lib/python3.5/dist-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


Char vectorization process Done!


## Features Aggregation

In [35]:
del all_comment_text
gc.collect()

0

In [36]:
train_tfidf_features = hstack([train_char_features, train_word_features]).tocsr()
test_tfidf_features = hstack([test_char_features, test_word_features]).tocsr()

In [37]:
del train_char_features, train_word_features, test_char_features, test_word_features
gc.collect()

0

## Build Vocabulary & Tokenizer

In [38]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='"#$%&()+,-./:;<=>@[\\]^_`{|}~\t\n')

In [39]:
print('Automatically train vocab & tokenizer...')
tokenizer.fit_on_texts(cleaned_train_comments + cleaned_test_comments)

train_sequences = tokenizer.texts_to_sequences(cleaned_train_comments)
test_sequences = tokenizer.texts_to_sequences(cleaned_test_comments)

word_index = tokenizer.word_index
print('Found %s unique tokens' % len(word_index))

train_data = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of train_data tensor: ', train_data.shape)
print('Shape of train_label tensor: ', train_labels.shape)

test_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of test_data tensor: ', test_data.shape)

Automatically train vocab & tokenizer...
Found 353757 unique tokens
Shape of train_data tensor:  (159571, 350)
Shape of train_label tensor:  (159571, 6)
Shape of test_data tensor:  (153164, 350)


## Pos Feature Extraction

In [40]:
def sent2pos(sentence):
    try:
        tag = TextBlob(sentence).tags
    except:
        print(sentence)
        
    updated_sentence = ' '.join([i[0] for i in tag])
    tagged = ' '.join([i[1] for i in tag])
    return updated_sentence, tagged

In [41]:
inverse_word_index = {v: k for k, v in word_index.items()}

In [42]:
Pos_comments = []
Pos_updated_sentence = []
for text in tqdm(train_sequences):
    text_ = ' '.join([inverse_word_index[word] for word in text])    # convert to word format
    if not isinstance(text_, str):
        print(text, '\n', text_)
    updated_sentence, tags = sent2pos(text_)
    Pos_updated_sentence.append(updated_sentence)
    Pos_comments.append(tags)
    assert len(updated_sentence.split(' ')) == len(tags.split(' ')), "T1 {} T2 {}".format(len(text), len(tags.split()))
    
Pos_test_comments = []
Pos_test_updated_sentence = []
for text in tqdm(test_sequences):
    text_ = ' '.join([inverse_word_index[word] for word in text])
    updated_test_sentence, test_tags = sent2pos(text_)
    Pos_test_updated_sentence.append(updated_test_sentence)
    Pos_test_comments.append(test_tags)
    assert len(updated_test_sentence.split(' ')) == len(test_tags.split(' ')), "T1 {} T2 {}".format(len(text), len(test_tags.split()))

100%|██████████| 153164/153164 [08:42<00:00, 292.94it/s]


In [43]:
pos_tokenizer = Tokenizer(num_words=50, filters='"#$%&()+,-./:;<=>@[\\]^_`{|}~\t\n')

In [44]:
print('Automatically train pos tokenizer...')
pos_tokenizer.fit_on_texts(Pos_comments + Pos_test_comments)

train_pos_sequences = pos_tokenizer.texts_to_sequences(Pos_comments)
test_pos_sequences = pos_tokenizer.texts_to_sequences(Pos_test_comments)

pos_word_index = pos_tokenizer.word_index
print('Found %s unique tokens' % len(pos_word_index))

pos_train_data = pad_sequences(train_pos_sequences, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', pos_train_data.shape)

pos_test_data = pad_sequences(test_pos_sequences, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of test_data tensor:', pos_test_data.shape)

Automatically train pos tokenizer...
Found 34 unique tokens
Shape of data tensor: (159571, 350)
Shape of test_data tensor: (153164, 350)


## Second time valid for tokenzier

In [45]:
print('Automatically train pos tokenizer secondly...')
cleaned_train_comments, cleaned_test_comments = [], []
for text in Pos_updated_sentence:
    cleaned_train_comments.append(clean_datasets(text))
for text in Pos_test_updated_sentence:
    cleaned_test_comments.append(clean_datasets(text))
    
train_sequences = tokenizer.texts_to_sequences(cleaned_train_comments)
test_sequences = tokenizer.texts_to_sequences(cleaned_test_comments)

word_index = tokenizer.word_index
print('Found %s unique tokens' % len(word_index))

train_data = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of train_data tensor:', train_data.shape)
print('Shape of train_label tensor:', train_labels.shape)

test_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of test_data tensor:', test_data.shape)

Automatically train pos tokenizer secondly...
Found 353757 unique tokens
Shape of train_data tensor: (159571, 350)
Shape of train_label tensor: (159571, 6)
Shape of test_data tensor: (153164, 350)


In [46]:
train_df['cleaned_comment_text'] = cleaned_train_comments
test_df['cleaned_comment_text'] = cleaned_test_comments
train_df.to_csv(PATH + 'cleaned_train.csv', index=False)
test_df.to_csv(PATH + 'cleaned_test.csv', index=False)

In [47]:
del cleaned_train_comments, cleaned_test_comments
gc.collect()

14

## Sentence Embedding (Build a matrix)

In [48]:
print('Preparing embedding matrix...')
nb_words = min(MAX_NB_WORDS, len(word_index))
embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))

null_words = open(PATH + 'null_words.txt', 'w', encoding='utf-8')

for word, idx in word_index.items():
    if idx >= MAX_NB_WORDS:
        null_words.write(word + ', ' + str(word_count_dict[word]) + '\n')
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[idx] = embedding_vector
    else:
        null_words.write(word + ', ' + str(word_count_dict[word]) + '\n')
print('Null_word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

Preparing embedding matrix...
Null_word embeddings: 26629


In [49]:
print('Sorting null_words...')
null_dict = {}
with open(PATH + 'null_words.txt', 'r', encoding='utf-8') as nullword:
    for line in nullword:
        word, count = line.strip('\n').split(', ')
        null_dict[word] = int(count)

null_dict = sorted(null_dict.items(), key=operator.itemgetter(1), reverse=True)

with open(PATH + 'null_words.txt', 'w', encoding='utf-8') as output:
    for word, count in null_dict:
        output.write(word + ', ' + str(count) + '\n')
print('Sorting operation Done!')

Sorting null_words...
Sorting operation Done!


# Model Training

## Baseline Model with Using Tfidf Features

In [ ]:
kfold = KFold(n_splits=FOLD_COUNT, shuffle=False)
tfidf_models = []
for i, (train_idx, test_idx) in enumerate(kfold.split(train_tfidf_features)):
    print('## In fold {} ##'.format(i + 1))
    classifier = LogisticRegression(solver='sag', C=12.0)
    
    for class_name in list_classes:
        print('Processing {} ...'.format(class_name))
        train_target = train_df[class_name][train_idx]
        
        classifier.fit(train_tfidf_features[train_idx], train_target)
        y_pred = classifier.predict(train_tfidf_features[train_idx])
        
        print('Training accuracy is {}'.format(accuracy_score(y_pred, train_target)))
        
        val_target = train_df[class_name][test_idx]
        val_pred = classifier.predict(train_tfidf_features[test_idx])
        
        print('Validation accuracy is {}'.format(accuracy_score(val_pred, val_target)))
        
    tfidf_models.append(classifier)
print('K-fold cross validation Done!')

## Logistic Regression

In [ ]:
for i, model in enumerate(tfidf_models):
    print('## In Model {} ##'.format(i + 1))
    # predictions = {'id': test_df['id']}
    predictions = OrderedDict()
    predictions['id'] = test_df['id']
    
    for class_name in list_classes:
        predictions[class_name] = model.predict_proba(test_tfidf_features)[:, 1]
        print('Predict the proba for {} Done!'.format(class_name))
        print(predictions.keys())
    
    print(predictions.keys())
    submission = pd.DataFrame.from_dict(predictions)
    submission.to_csv('results/LR_Based/Logistic_Regression_Submission_{}.csv'.format(i), index=False)

In [ ]:
result_list = ['results/LR_Based/Logistic_Regression_Submission_{}.csv'.format(i) for i in range(0, FOLD_COUNT)]
bagging(result_list, 'results/LR_Based/bagging.csv')
print('Bagging operation Done!')

## ExtraTreeClassifier

In [ ]:
accs = []
# et_predictions = {'id': test_df['id']}
et_predictions = OrderedDict()
et_predictions['id'] = test_df['id']

for class_name in list_classes:
    train_target = train_df[class_name]
    classifier = ExtraTreesClassifier(n_estimators=30)
    
    cv_score = np.mean(cross_val_score(classifier, train_tfidf_features, train_target, cv=10, scoring='roc_auc'))
    accs.append(cv_score)
    print('CV Score for class {} is {}'.format(class_name, cv_score))
    
    classifier.fit(train_tfidf_features, train_target)
    et_predictions[class_name] = classifier.predict_proba(test_tfidf_features)[:, 1]
    
submission = pd.DataFrame.from_dict(et_predictions)
submission.to_csv('result/LR_Based/ExtraTreesClassifier_Submission.csv', index=False)

## Light-GBM For Ensembled_Datasets

In [ ]:
def fit_every_feature_model(feature_data, label, feature_name, feature_test_data, fold_count, predict=False):
    predictions = np.zeros(shape=(len(feature_test_data)))
    fold_size = len(feature_data) // fold_count
    
    print('Feature name is {}'.format(feature_name))
    auc_score = 0
    for fold_id in range(0, fold_count):
        print('## Fold In {} ##'.format(fold_id + 1))
        fold_start = fold_size * fold_id
        fold_end = fold_start + fold_size
        if fold_id == fold_size - 1:
            fold_end = len(feature_data)
            
        train_x = np.concatenate((feature_data[:fold_start], feature_data[fold_end:]))
        train_y = np.concatenate((label[:fold_start], label[fold_end:]))
        
        val_x = feature_data[fold_start:fold_end]
        val_y = label[fold_start:fold_end]
        
        lgb_train = lgb.Dataset(train_x, train_y)
        lgb_val = lgb.Dataset(val_x, val_y)
        
        lgbm_model = lgb.LGBMClassifier(max_depth=5, metric='auc', n_estimators=10000, num_leaves=32, boosting_type='gbdt', \
                                       learning_rate=0.01, feature_fraction=0.3, bagging_fraction=0.8, bagging_freq=5, reg_lambda=0)
        lgbm_model.fit(X=train_x, y=train_y, eval_metric=['auc', 'binary_logloss'], eval_set=(val_x, val_y), early_stopping_rounds=1000, verbose=500)
        auc_score += lgbm_model.best_score_['valid_0']['auc']
        lgb.plot_importance(lgbm_model, max_num_features=30)
        plt.show()
        if predict:
            prediction = lgbm_model.predict_proba(feature_test_data)[:, 1]
            predictions += prediction
            del lgbm_model
    predictions /= fold_count
    print('Training LightGBM Done!')
    return predictions, auc_score / fold_count

In [ ]:
predictions, auc_scores = [], []
for i, feature_name in enumerate(list_classes):
    prediction, auc = fit_every_feature_model(train_ensembled_data[i], train_df[feature_name].values, feature_name, test_ensembled_data[i], 10, predict=True)
    auc_scores.append(auc)
    predictions.append(prediction)
print('Overall AUC Score is {}'.format(sum(auc_scores) / 6))
print('For each:'.format(auc_scores))

In [ ]:
submission = pd.read_csv('datasets/sample_submission.csv')
for i, feature_name in enumerate(list_classes):
    submission[feature_name] = predictions[i]
submission.to_csv('results/LightGBM/LightGBM_with_Meta_data.csv', index=False)

## Training Styles

In [67]:
STAMP = 'model_pool/av_rnn/pavel_rnn_%.2f_%.2f'%(0.5, 0.5)

In [66]:
def _train_model_by_auc(model, batch_size, train_x, train_y, val_x, val_y):
    best_auc = -1
    best_weight = None
    best_epoch = 0
    current_epoch = 1
    
    while True:
        model.fit(train_x, train_y, batch_size=batch_size, epoch=1, validation_data=[val_x, val_y])
        y_pred = model.predict(val_x, batch_size=batch_size)
        current_auc = roc_auc_score(val_y, y_pred)
        print('Epoch {} auc {:.6f} best_auc {:.6f}'.format(current_epoch, current_auc, best_auc))
        current_epoch += 1
        if best_auc < current_auc or best_auc == -1:
            best_auc = current_auc
            best_weight = model.get_weights()
            best_epoch = current_epoch
        else:
            # early stop
            if current_epoch - best_epoch == 5:
                break
                
    model.set_weights(best_weights)
    return model, best_auc

def _train_model_by_logloss(model, batch_size, train_x, pos_train_x, train_y, val_x, pos_val_x, val_y, fold_id):
    early_stopping = EarlyStopping(monitor='val_loss', patience=7)
    best_model_path = STAMP + str(fold_id) + '.h5'
    model_checkpoint = ModelCheckpoint(best_model_path, save_best_only=True, save_weights_only=True)
    train_data = {'Onehot': train_x, 'POS': pos_train_x}
    val_data = {'Onehot': val_x, 'POS': pos_val_x}
    
    hist = model.fit(train_data, train_y, validation_data=(val_data, val_y), epochs=50, batch_size=batch_size, shuffle=True, callbacks=[early_stopping, model_checkpoint])
    best_val_score = min(hist.history['val_loss'])
    predictions = model.predict(val_data)
    auc = roc_auc_score(val_y, predictions)
    print('AUC Score', auc)
    return model, best_val_score, auc, predictions

def train_folds(x, pos_x, y, fold_count, batch_size, get_model_func):
    fold_size = len(x) // fold_count
    models = []
    fold_predictions = []
    score, total_auc = 0, 0
    for fold_id in range(0, fold_count):
        fold_start = fold_size * fold_id
        fold_end = fold_start + fold_size
        
        if fold_id == fold_count - 1:
            fold_end = len(x)
            
        train_x = np.concatenate((x[:fold_start], x[fold_end:]))
        train_y = np.concatenate((y[:fold_start], y[fold_end:]))
        
        val_x = x[fold_start: fold_end]
        val_y = y[fold_start: fold_end]
        
        pos_train_x = np.concatenate((pos_x[:fold_start], pos_x[fold_end:]))
        pos_val_x = pos_x[fold_start: fold_end]
        print('## In fold {} ## : '.format(fold_id + 1))
        model, best_val_score, auc, fold_prediction = _train_model_by_logloss(get_model_func, batch_size, train_x, pos_train_x, train_y, val_x, pos_val_x, val_y, fold_id)
        
        score += best_val_score
        total_auc += auc
        fold_predictions.append(fold_prediction)
        models.append(model)
    return models, score / fold_count, total_auc / fold_count , fold_predictions

## Model Zoo

In [69]:
class AttentionWeightedAverage(Layer):
    """
    Model for computing a weighted average of the different channels across timesteps.
    Uses 1 parameter per channel to compute the attention value for a single timestep.
    """
    def __init__(self, return_attention=False, **kwargs):
        self.init = initializers.get('uniform')
        self.supports_masking = True
        self.return_attention = return_attention
        super(AttentionWeightedAverage, self).__init__(**kwargs)
        
    def build(self, input_shape):
        self.input_spec = [InputSpec(ndim=3)]
        assert len(input_shape) == 3
        
        self.W = self.add_weight(shape=(input_shape[2], 1),
                                 name='{}_W'.format(self.name),
                                 initializer=self.init)
        
        self.trainable_weights = [self.W]
        super(AttentionWeightedAverage, self).build(input_shape)
        
    def call(self, x, mask=None):
        logits = K.dot(x, self.W)
        x_shape = K.shape(x)
        logits = K.reshape(logits, (x_shape[0], x_shape[1]))
        ai = K.exp(logits - K.max(logits, axis=-1, keepdims=True))
        
        if mask is not None:
            mask = K.cast(mask, K.floatx())
            ai = ai * mask
        attn_weights = ai / (K.sum(ai, axis=1, keepdims=True) + K.epsilon())
        weighted_input = x * K.expand_dims(attn_weights)
        result = K.sum(weighted_input, axis=1)
        if self.return_attention:
            return [result, attn_weights]
        return result
    
    def get_output_shape_for(self, input_shape):
        return self.compute_output_shape(input_shape)
    
    def compute_output_shape(self, input_shape):
        output_len = input_shape[2]
        if self.return_attention:
            return [(input_shape[0], output_len), (input_shape[0], input_shape[1])]
        return (input_shape[0], output_len)
    
    def compute_mask(self, input, input_mask=None):
        if isinstance(input_mask, list):
            return [None] * len(input_mask)
        else:
            return None

In [70]:
class KMaxPooling(Layer):
    """
    K-max pooling layer that extracts the k-highest activations from a sequence (2nd dimension).
    With TensorFlow Backend.
    """
    def __init__(self, k=1, **kwargs):
        super().__init__(**kwargs)
        self.input_spec = InputSpec(ndim=3)
        self.k = k
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[2] * self.k)
    
    def call(self, inputs):
        # top_k function can only be applied along the last dimension
        shifted_input = tf.transpose(inputs, [0, 2, 1])
        top_k = tf.nn.top_k(shifted_input, self.k, True, None)[0]
        return Flatten()(top_k)

In [71]:
def get_av_cnn(nb_words, embedding_dim, embedding_matrix, max_sequence_length, out_size):
    embedding_layer = Embedding(nb_words,
                                embedding_dim,
                                weights=[embedding_matrix],
                                input_length=max_sequence_length,
                                trainable=False)
    
    filter_nums = 300
    input_layer = Input(shape=(max_sequence_length,), dtype='int32', name='Onehot')
    embedding_sequences = embedding_layer(input_layer)
    final_embedding_sequences = SpatialDropout1D(0.25)(embedding_sequences)
    
    conv_0 = Conv1D(filter_nums, 1, kernel_initializer='normal', padding='valid', activation='relu')(final_embedding_sequences)
    conv_1 = Conv1D(filter_nums, 2, kernel_initializer='normal', padding='valid', activation='relu')(final_embedding_sequences)
    conv_2 = Conv1D(filter_nums, 3, kernel_initializer='normal', padding='valid', activation='relu')(final_embedding_sequences)
    conv_3 = Conv1D(filter_nums, 4, kernel_initializer='normal', padding='valid', activation='relu')(final_embedding_sequences)
    
    attn_0 = AttentionWeightedAverage()(conv_0)
    avg_0 = GlobalAveragePooling1D()(conv_0)
    maxpool_0 = GlobalMaxPooling1D()(conv_0)
    
    attn_1 = AttentionWeightedAverage()(conv_1)
    avg_1 = GlobalAveragePooling1D()(conv_1)
    maxpool_1 = GlobalMaxPooling1D()(conv_1)
    
    attn_2 = AttentionWeightedAverage()(conv_2)
    avg_2 = GlobalAveragePooling1D()(conv_2)
    maxpool_2 = GlobalMaxPooling1D()(conv_2)
    
    attn_3 = AttentionWeightedAverage()(conv_3)
    avg_3 = GlobalAveragePooling1D()(conv_3)
    maxpool_3 = GlobalMaxPooling1D()(conv_3)
    
    merged_maxpool = merge([maxpool_0, maxpool_1, maxpool_2, maxpool_3], mode='concat', concat_axis=1)
    merged_attn = merge([attn_0, attn_1, attn_2, attn_3], mode='concat', concat_axis=1)
    merged_avg = merge([avg_0, avg_1, avg_2, avg_3], mode='concat', concat_axis=1)
    merged_tensor = merge([merged_maxpool, merged_attn, merged_avg], mode='concat', concat_axis=1)
    
    output = Dropout(0.7)(merged_tensor)
    output = Dense(units=144, activation='relu')(output)
    output = Dense(units=out_size, activation='sigmoid')(output)
    
    model = Model(inputs=input_layer, outputs=output)
    adam_optimizer = optimizers.Adam(lr=1e-3, decay=1e-6)
    model.compile(loss='binary_crossentropy', optimizer=adam_optimizer, metrics=['accuracy'])
    return model

In [72]:
def get_av_pos_cnn(nb_words, embedding_dim, embedding_matrix, max_sequence_length, out_size):
    embedding_layer = Embedding(nb_words, 
                                embedding_dim, 
                                weights=[embedding_matrix], 
                                input_length=max_sequence_length, 
                                trainable=False)
    
    pos_embedding_layer = Embedding(50,
                                    30,
                                    input_length=max_sequence_length,
                                    trainable=True)
    
    filter_nums = 325
    input_layer = Input(shape=(max_sequence_length,), dtype='int32', name='Onehot')
    pos_input_layer = Input(shape=(max_sequence_length,), dtype='int32', name='POS')
    
    embedding_sequences = embedding_layer(input_layer)
    pos_sequences = pos_embedding_layer(pos_input_layer)
    merged_embedding_layer = concatenate([embedding_sequences, pos_sequences])
    final_embedding_sequences = SpatialDropout1D(0.25)(embedding_layer)
    
    conv_0 = Conv1D(filter_nums, 1, kernel_initializer='normal', padding='valid', activation='relu')(final_embedding_sequences)
    conv_1 = Conv1D(filter_nums, 2, kernel_initializer='normal', padding='valid', activation='relu')(final_embedding_sequences)
    conv_2 = Conv1D(filter_nums, 3, kernel_initializer='normal', padding='valid', activation='relu')(final_embedding_sequences)
    conv_3 = Conv1D(filter_nums, 4, kernel_initializer='normal', padding='valid', activation='relu')(final_embedding_sequences)

    attn_0 = AttentionWeightedAverage()(conv_0)
    avg_0 = GlobalAveragePooling1D()(conv_0)
    maxpool_0 = GlobalMaxPooling1D()(conv_0)
    
    attn_1 = AttentionWeightedAverage()(conv_1)
    avg_1 = GlobalAveragePooling1D()(conv_1)
    maxpool_1 = GlobalMaxPooling1D()(conv_1)
    
    attn_2 = AttentionWeightedAverage()(conv_2)
    avg_2 = GlobalAveragePooling1D()(conv_2)
    maxpool_2 = GlobalMaxPooling1D()(conv_2)
    
    attn_3 = AttentionWeightedAverage()(conv_3)
    avg_3 = GlobalAveragePooling1D()(conv_3)
    maxpool_3 = GlobalMaxPooling1D()(conv_3)
    
    merged_tensor_maxpool = merge([maxpool_0, maxpool_1, maxpool_2, maxpool_3], mode='concat', concat_axis=1)
    merged_tensor_attn = merge([attn_0, attn_1, attn_2, attn_3], mode='concat', concat_axis=1)
    merged_tensor_avg = merge([avg_0, avg_1, avg_2, avg_3], mode='concat', concat_axis=1)
    merged_tensor = merge([merged_tensor_maxpool, merged_tensor_attn, merged_tensor_avg], mode='concat', concat_axis=1)
    
    output = Dropout(0.7)(merged_tensor)
    output = Dense(units=144, activation='relu')(output)
    output = Dense(units=out_size, activation='sigmoid')(output)
    
    model = Model(inputs=[input_layer, pos_input_layer], outputs=output)
    adam_optimizer = optimizers.Adam(lr=1e-3, decay=1e-6, clipvalue=5)
    model.compile(loss='binary_corssentropy', optimizer=adam_optimizer, matrics=['accuracy'])
    return model

In [73]:
def get_kmax_text_cnn(nb_words, embedding_dim, embedding_matrix, max_sequence_length, out_size):
    embedding_layer = Embedding(nb_words,
                                embedding_dim,
                                weights=[embedding_matrix],
                                input_length=max_sequence_length,
                                trainable=False)
    
    filter_nums = 180
    
    input_layer = Input(shape=(max_sequence_length,), dtype='int32', name='Onehot')
    embedding_sequences = embedding_layer(input_layer)
    final_embedding_sequences = SpatialDropout1D(embedding_sequences)
    
    conv_0 = Conv1D(filter_nums, 1, kernel_initializer='normal', padding='valid', activation='relu')(final_embedding_sequences)
    conv_1 = Conv1D(filter_nums, 2, kernel_initializer='normal', padding='valid', activation='relu')(final_embedding_sequences)
    conv_2 = Conv1D(filter_nums, 3, kernel_initializer='normal', padding='valid', activation='relu')(final_embedding_sequences)
    conv_3 = Conv1D(filter_nums, 4, kernel_initializer='normal', padding='valid', activation='relu')(final_embedding_sequences)
    
    maxpool_0 = KMaxPooling(k=3)(conv_0)
    maxpool_1 = KMaxPooling(k=3)(conv_1)
    maxpool_2 = KMaxPooling(k=3)(conv_2)
    maxpool_3 = KMaxPooling(k=3)(conv_3)
    
    merged_tensor = merge([maxpool_0, maxpool_1, maxpool_2, maxpool_3], mode='concat', concat_axis=1)
    output = Dropout(0.6)(merged_tensor)
    output = Dense(units=144, activation='relu')(output)
    output = Dense(units=out_size, activation='sigmoid')(output)
    
    model = Model(inputs=input_layer, outputs=output)
    adam_optimizer = optimizers.Adam(lr=1e-3, decay=1e-6)
    model.compile(loss='binary_crossentropy', optimizer=adam_optimizer, metrics=['accuracy'])
    return model

In [74]:
def get_rcnn(nb_words, embedding_dim, embedding_matrix, max_sequence_length, out_size):
    embedding_layer = Embedding(nb_words,
                                embedding_dim,
                                weights=[embedding_matrix],
                                input_length=max_sequence_length,
                                trainable=False)
    
    recurrent_units = 64
    filter_nums = 128
    
    input_layer = Input(shape=(max_sequence_length,), dtype='int32', name='Onehot')
    embedding_sequences = embedding_layer(input_layer)
    final_embedding_sequences = SpatialDropout1D(0.2)(embedding_sequences)
    
    rnn_layer = Bidirectional(CuDNNGRU(recurrent_units, return_sequences=True))(final_embedding_sequences)
    conv_layer = Conv1D(filter_nums, 2, kernel_initializer='normal', padding='valid', activation='relu', strides=1)(rnn_layer)
    
    maxpool = GlobalMaxPooling1D()(conv_layer)
    attn = AttentionWeightedAverage()(conv_layer)
    avg = GlobalAveragePooling1D()(conv_layer)
    
    merged_tensor = merge([maxpool, attn, avg], mode='concat', concat_axis=1)
    output = Dropout(0.5)(merged_tensor)
    output = Dense(units=120, activation='relu')(output)
    output = Dense(units=out_size, activation='sigmoid')(output)
    
    model = Model(inputs=input_layer, outputs=output)
    adam_optimizer = optimizers.Adam(lr=1e-3, decay=1e-6, clipvalue=5)
    model.compile(loss='binary_crossentropy', optimizer=adam_optimizer, metrics=['accuracy'])
    return model

In [83]:
def get_av_rnn(nb_words, embedding_dim, embedding_matrix, max_sequence_length, out_size):
    embedding_layer = Embedding(nb_words,
                                embedding_dim,
                                weights=[embedding_matrix],
                                input_length=max_sequence_length,
                                trainable=False)
    
    recurrent_units = 64
    
    input_layer = Input(shape=(max_sequence_length,), dtype='int32', name='Onehot')
    embedding_sequences = embedding_layer(input_layer)
    final_embedding_sequences = SpatialDropout1D(0.25)(embedding_sequences)
    
    rnn_layer_0 = Bidirectional(CuDNNGRU(recurrent_units, return_sequences=True))(final_embedding_sequences)
    rnn_layer_1 = Bidirectional(CuDNNGRU(recurrent_units, return_sequences=True))(rnn_layer_0)
    merged_rnn_layer = merge([rnn_layer_0, rnn_layer_1], mode='concat', concat_axis=2)
    
    last_layer = Lambda(lambda t: t[:, -1], name='last_layer')(merged_rnn_layer)
    maxpool = GlobalMaxPooling1D()(merged_rnn_layer)
    attn = AttentionWeightedAverage()(merged_rnn_layer)
    avg = GlobalAveragePooling1D()(merged_rnn_layer)
    
    merged_tensor = merge([last_layer, maxpool, attn, avg], mode='concat', concat_axis=1)
    output = Dropout(0.5)(merged_tensor)
    output = Dense(units=144, activation='relu')(output)
    output = Dense(units=out_size, activation='sigmoid')(output)
    
    model = Model(inputs=input_layer, outputs=output)
    adam_optimizer = optimizers.Adam(lr=1e-3, decay=1e-6, clipvalue=5)
    model.compile(loss='binary_crossentropy', optimizer=adam_optimizer, metrics=['accuracy'])
    return model

In [76]:
def get_av_pos_rnn(nb_words, embedding_dim, embedding_matrix, max_sequence_length, out_size):
    embedding_layer = Embedding(nb_words,
                                embedding_dim,
                                weights=[embedding_matrix],
                                input_length=max_sequence_length,
                                trainable=False)
    
    pos_embedding_layer = Embedding(50,
                                    35,
                                    input_length=max_sequence_length,
                                    trainable=True)
    
    recurrent_units = 64
    
    input_layer = Input(shape=(max_sequence_length,), dtype='int32', name='Onehot')
    pos_input_layer = Input(shape=(max_sequence_length,), dtype='int32', name='POS')
    embedding_sequences = embedding_layer(input_layer)
    pos_sequences = pos_embedding_layer(pos_input_layer)
    merged_embedding_layer = concatenate([embedding_sequences, pos_sequences], axis=2)
    final_embedding_sequences = SpatialDropout1D(0.2)(merged_embedding_layer)
    
    rnn_layer_0 = Bidirectional(CuDNNGRU(recurrent_units, return_sequences=True))(final_embedding_sequences)
    rnn_layer_0 = SpatialDropout1D(0.3)(rnn_layer_0)
    rnn_layer_1 = Bidirectional(CuDNNGRU(recurrent_units, return_sequences=True))(rnn_layer_0)
    
    last_layer = Lambda(lambda t: t[:, -1], name='last_layer')(rnn_layer_1)
    maxpool = GlobalMaxPooling1D()(rnn_layer_1)
    attn = AttentionWeightedAverage()(rnn_layer_1)
    avg = GlobalAveragePooling1D()(rnn_layer_1)
    
    merged_tensor = merge([last_layer, maxpool, attn, avg], mode='concat', concat_axis=1)
    output = Dropout(0.5)(merged_tensor)
    output = Dense(units=144, activation='relu')(output)
    output = Dense(units=out_size, activation='sigmoid')(output)
    
    model = Model(inputs=[input_layer, pos_input_layer], outputs=output)
    adam_optimizer = optimizers.Adam(lr=1e-3, decay=1e-6, clipvalue=5)
    model.compile(loss='binary_crossentropy', optimizer=adam_optimizer, metrics=['accuracy'])
    return model

In [77]:
def get_dropout_bigru(nb_words, embedding_dims, embedding_matrix, max_sequence_length, out_size):
    embedding_layer = Emebdding(nb_words,
                                embedding_dims,
                                weights=[embedding_matrix],
                                input_length=max_sequence_length,
                                trainable=False)
    
    recurrent_units = 64
    
    input_layer = Input(shape=(max_sequence_length,), dtype='int32', name='Onehot')
    embedding_sequences = embedding_layer(input_layer)
    final_embedding_sequences = SpatialDropout1D(0.2)(embedding_sequences)
    
    rnn_layer = Bidirectional(CuDNNGRU(recurrent_units, return_sequences=True))(final_embedding_sequences)
    rnn_layer = Dropout(0.35)(rnn_layer)
    rnn_layer = Bidirectional(CuDNNGRU(recurrent_units, return_sequences=True))(rnn_layer)
    
    last_layer = Lambda(lambda t: t[:, -1])(rnn_layer)
    maxpool = GlobalMaxPooling1D()(rnn_layer)
    avg = GlobalAveragePooling1D()(rnn_layer)
    
    merged_tensor = merge([last_layer, maxpool, avg], mode='concat', concat_axis=1)
    output = Dropout(0.5)(merged_tensor)
    output = Dense(units=72, activation='relu')(output)
    output = Dense(units=out_size, activation='sigmoid')(output)
    
    model = Model(inputs=input_layer, outputs=output)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

## Start Training

In [84]:
model_name = 'fasttext-avrnn-' + str(nb_words) + 'vocabulary-' + str(MAX_SEQUENCE_LENGTH) + 'length'
model = get_av_rnn(nb_words, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, 6)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Onehot (InputLayer)             (None, 350)          0                                            
__________________________________________________________________________________________________
embedding_7 (Embedding)         (None, 350, 300)     30000000    Onehot[0][0]                     
__________________________________________________________________________________________________
spatial_dropout1d_7 (SpatialDro (None, 350, 300)     0           embedding_7[0][0]                
__________________________________________________________________________________________________
bidirectional_11 (Bidirectional (None, 350, 128)     140544      spatial_dropout1d_7[0][0]        
__________________________________________________________________________________________________
bidirectio

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:16: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  app.launch_new_instance()
/usr/local/lib/python3.5/dist-packages/keras/legacy/layers.py:465: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:23: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [87]:
models, val_loss, total_auc, fold_predictions = train_folds(train_data, pos_train_data, train_labels, FOLD_COUNT, BATCH_SIZE, model)

## In fold 1 ## : 
Train on 143614 samples, validate on 15957 samples
Epoch 1/50
143614/143614 [==============================] - 155s 1ms/step - loss: 0.0300 - acc: 0.9876 - val_loss: 0.0365 - val_acc: 0.9856
Epoch 2/50
143614/143614 [==============================] - 161s 1ms/step - loss: 0.0292 - acc: 0.9880 - val_loss: 0.0375 - val_acc: 0.9848
Epoch 3/50
143614/143614 [==============================] - 164s 1ms/step - loss: 0.0285 - acc: 0.9883 - val_loss: 0.0386 - val_acc: 0.9841
Epoch 4/50
143614/143614 [==============================] - 163s 1ms/step - loss: 0.0277 - acc: 0.9887 - val_loss: 0.0400 - val_acc: 0.9843
Epoch 5/50
143614/143614 [==============================] - 156s 1ms/step - loss: 0.0273 - acc: 0.9887 - val_loss: 0.0403 - val_acc: 0.9838
Epoch 6/50
143614/143614 [==============================] - 163s 1ms/step - loss: 0.0265 - acc: 0.9890 - val_loss: 0.0398 - val_acc: 0.9849
Epoch 7/50
143614/143614 [==============================] - 164s 1ms/step - loss: 0.0261 -

Epoch 7/50
143614/143614 [==============================] - 166s 1ms/step - loss: 0.0149 - acc: 0.9940 - val_loss: 0.0122 - val_acc: 0.9954
Epoch 8/50
143614/143614 [==============================] - 166s 1ms/step - loss: 0.0150 - acc: 0.9940 - val_loss: 0.0116 - val_acc: 0.9954
AUC Score 0.9991669972401808
## In fold 8 ## : 
Train on 143614 samples, validate on 15957 samples
Epoch 1/50
143614/143614 [==============================] - 166s 1ms/step - loss: 0.0155 - acc: 0.9939 - val_loss: 0.0069 - val_acc: 0.9977
Epoch 2/50
143614/143614 [==============================] - 166s 1ms/step - loss: 0.0153 - acc: 0.9939 - val_loss: 0.0076 - val_acc: 0.9976
Epoch 3/50
143614/143614 [==============================] - 166s 1ms/step - loss: 0.0150 - acc: 0.9940 - val_loss: 0.0090 - val_acc: 0.9971
Epoch 4/50
143614/143614 [==============================] - 166s 1ms/step - loss: 0.0147 - acc: 0.9941 - val_loss: 0.0106 - val_acc: 0.9961
Epoch 5/50
143614/143614 [==============================] - 1

In [88]:
print('Overall val-loss: {}, AUC {}'.format(val_loss, total_auc))

Overall val-loss: 0.013377069772156469, AUC 0.9977374191614368


# Prediction

In [89]:
submit_path_prefix = 'results/RNN_Based/' + model_name

print('Predicting testing results...')
test_predicts_list = []
for fold_id, model in enumerate(models):
    test_data_dict = {'Onehot': test_data, 'POS': pos_test_data}
    test_predict = model.predict(test_data_dict, batch_size=BATCH_SIZE, verbose=1)
    test_predicts_list.append(test_predict)
    np.save('parameters_pool/AVRNN/{}-AV-POS-RNN.npy'.format(fold_id), test_predict)
    
test_predicts = np.zeros(test_predicts_list[0].shape)
for fold_predict in test_predicts_list:
    test_predicts += fold_predict
test_predicts /= len(test_predicts_list)

test_ids = test_df['id'].values
test_ids = test_ids.reshape((len(test_ids), 1))

test_predicts = pd.DataFrame(data=test_predicts, columns=list_classes)
test_predicts['id'] = test_ids
test_predicts = test_predicts[['id'] + list_classes]
submit_path = submit_path_prefix + '-L{:4f}-A{:4f}.csv'.format(val_loss, total_auc)
test_predicts.to_csv(submit_path, index=False)

Predicting testing results...
153164/153164 [==============================] - 67s 438us/step


## OOB (Out-of-Bag) Evaluation

In [64]:
train_fold_predictions = np.concatenate((fold_predictions), axis=0)
train_auc = roc_auc_score(train_labels, train_fold_predictions)
print('Training AUC', train_auc)

Training AUC 0.9972381428584968


In [65]:
print('Predicting training results...')
train_ids = train_df['id'].values
train_ids = train_ids.reshape((len(train_ids), 1))

train_predicts = pd.DataFrame(data=train_fold_predictions, columns=list_classes)
train_predicts['id'] = train_ids
train_predicts = train_predicts[['id'] + list_classes]
submit_path = submit_path_prefix + '-(Train)-L{:4f}-A{:4f}.csv'.format(val_loss, train_auc)
train_predicts.to_csv(submit_path, index=False)
print('Done!')

Predicting training results...
Done!


# Result Ensemble (For Test Format)

## Bagging

In [ ]:
def bagging(arrs, path):
    print("Doing ensemble on")
    subs = []
    for arr in arrs:
        print(arr)
        subs.append(pd.read_csv(arr))
    
    for sub in subs[1:]:
        for c in list_classes:
            subs[0][c] += sub[c]
    
    for c in list_classes:
        subs[0][c] /= len(subs)
        
    subs[0].to_csv(path, index=False)

# Check Correlation

In [ ]:
def check_corr(arr1, arr2):
    res = 0
    for col in arr1.columns.values[1:]:
        cur = arr1[col].corr(arr2[col])
        corr = (arr1[col].rank() / len(arr1)).corr(arr2[col].rank() / len(arr2))
        print(col, corr)
        res += corr
    print("Avg Rank: ", res / len(arr1.columns.values[1:]))

# Result Estimation (For Train Format)

In [ ]:
train_df